In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [127]:
from scipy import sparse
from scipy.sparse import coo_matrix 
import numpy as np

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [128]:
A_data = [39, 42, 15, 20]
A_row_indices = [0, 1, 1, 2,1]
A_col_indices = [0, 1, 2, 1,0]

B_data =  [9, 32, 51, 24]
B_row_indices =  [0, 2, 1, 2,1]
B_col_indices = [0, 2, 2, 1,0]

# Matrix dimensions
A_rows = 4
A_cols = 4
B_cols = 4
A_rows = 4

C_data = []
C_row_indices = []
C_col_indices = []

for i in range(len(A_data)):
    row_A = A_row_indices[i]
    col_A = A_col_indices[i]
    value_A = A_data[i]
    
    for j in range(len(B_data)):
        row_B = B_row_indices[j]
        col_B = B_col_indices[j]
        value_B = B_data[j]
        
        
        if col_A == row_B:
            C_data.append(value_A * value_B)
            C_row_indices.append(row_A)
            C_col_indices.append(col_B)


C = (C_data, (C_row_indices, C_col_indices))
C = coo_matrix((C_data, (C_row_indices, C_col_indices)), shape=(4, 4))
print(type(C))
print('-------------------------------------------------------')
print(C.toarray())

<class 'scipy.sparse._coo.coo_matrix'>
-------------------------------------------------------
[[ 351    0    0    0]
 [   0  360 2622    0]
 [   0    0 1020    0]
 [   0    0    0    0]]


implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [129]:
A = np.array([[2,0, 32,0], [0,0, 13, 0], [0,48, 0, 35],[0,0, 0, 59]])
B = np.array([[6, 0,0,0], [0,17,0,0], [0,0,87, 99],[0,41, 0,0]])

A_rows, A_cols = np.nonzero(A)
A_vals = A[A_rows, A_cols]
A_indptr = np.concatenate(([0], np.cumsum(np.bincount(A_rows, minlength=A.shape[0]))))

B_rows, B_cols = np.nonzero(B)
B_vals = B[B_rows, B_cols]
B_indptr = np.concatenate(([0], np.cumsum(np.bincount(B_rows, minlength=B.shape[0]))))

C_rows = []
C_cols = []
C_vals = []

for i in range(A.shape[0]):
    row_start = A_indptr[i]
    row_end = A_indptr[i+1]
    for j in range(B.shape[1]):
        col_start = B_indptr[j]
        col_end = B_indptr[j+1]
        dot_prod = 0
        k1, k2 = row_start, col_start
        while k1 < row_end and k2 < col_end:
            if A_cols[k1] == B_rows[k2]:
                dot_prod += A_vals[k1] * B_vals[k2]
                k1 += 1
                k2 += 1
            elif A_cols[k1] < B_rows[k2]:
                k1 += 1
            else:
                k2 += 1
        if dot_prod != 0:
            C_rows.append(i)
            C_cols.append(j)
            C_vals.append(dot_prod)

            
C = coo_matrix((C_vals, (C_rows, C_cols)), shape=(4, 4))
print(type(C))
print('-------------------------------------------------------')
print(C.toarray())

<class 'scipy.sparse._coo.coo_matrix'>
-------------------------------------------------------
[[  12    0 2784    0]
 [   0    0 1131    0]
 [   0  816    0 1435]
 [   0    0    0 2419]]


implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [178]:
import numpy as np
from scipy.sparse import csr_matrix

def solve_lin_sys(A_csr, b):
    x = np.zeros(A_csr.shape[0])
    r = b - A_csr.dot(x)
    p = r
    tol = 1e-6
    max_iter = 1000

    for i in range(max_iter):
        alpha = np.dot(r, r) / np.dot(p, A_csr.dot(p))
        x = x + alpha * p
        r_new = r - alpha * A_csr.dot(p)
        if np.linalg.norm(r_new) < tol:
            break
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        p = r_new + beta * p
        r = r_new
    return x

In [179]:
indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
b = np.array([0,1,1])
A_csr = csr_matrix((data, indices, indptr), shape=(3, 3))
print(type(A_csr))
print('--------------------------------------------------------')
print(A_csr.toarray())
print('--------------------------------------------------------')
print('x = ',solve_lin_sys(A_csr, b))

<class 'scipy.sparse._csr.csr_matrix'>
--------------------------------------------------------
[[1 0 2]
 [0 0 3]
 [4 5 6]]
--------------------------------------------------------
x =  [  30.1231933  -137.11129276    0.88181378]
